In [47]:
import torch
import pickle

# Load data
pseudodata_path = "/Users/xaviermootoo/Documents/Data/ssl-seizure-detection/pseudolabeled/jh101_12s_7min_np.pkl"
pdata = pickle.load(open(pseudodata_path, "rb"))

In [27]:
# List is of the form [[graph_1, graph_2], label] where each graph = [A, x, egde_attr]

print(len(pdata[0]))
print(len(pdata[0][0]))
print(len(pdata[0][0][0]))
print(pdata[0][0][0][0].shape)

2
2
3
(107, 107)


In [30]:
# Convert each entry [[[A, NF, EF'], [A, NF, EF']], Y] to tensors
pdata_t = []
for i in range(len(pdata)):
    graph_1 = [torch.from_numpy(a) for a in pdata[i][0][0]]
    graph_2 = [torch.from_numpy(a) for a in pdata[i][0][1]]
    label = torch.tensor([pdata[i][1]])
    pdata_t.append([graph_1, graph_2, label])

In [41]:
# The first adjacency matrix of the first graph_1
pdata_t[0][0][0]

tensor([[ 2.4411,  0.6607, -0.1258,  ...,  0.8382, -1.5899, -0.8058],
        [ 0.6607,  2.4411,  1.6718,  ...,  0.6053, -2.1600, -1.9578],
        [-0.1258,  1.6718,  2.4411,  ...,  0.4384, -1.4936, -1.5496],
        ...,
        [ 0.8382,  0.6053,  0.4384,  ...,  2.4411, -1.4207, -1.2728],
        [-1.5899, -2.1600, -1.4936,  ..., -1.4207,  2.4411,  1.8921],
        [-0.8058, -1.9578, -1.5496,  ..., -1.2728,  1.8921,  2.4411]],
       dtype=torch.float64)

In [55]:
# The first edge_attr of the first graph_1
print(pdata_t[0][0][2][0].shape)

torch.Size([107, 1])


In [69]:
import numpy as np

# Credit: https://github.com/pyg-team/pytorch_geometric/issues/964

def build_edge_idx(num_nodes):
    # Initialize edge index matrix
    E = torch.zeros((2, num_nodes * (num_nodes - 1)), dtype=torch.long)
    
    # Populate 1st row
    for node in range(num_nodes):
        for neighbor in range(num_nodes - 1):
            E[0, node * (num_nodes - 1) + neighbor] = node

    # Populate 2nd row
    neighbors = []
    for node in range(num_nodes):
        neighbors.append(list(np.arange(node)) + list(np.arange(node+1, num_nodes)))
    E[1, :] = torch.Tensor([item for sublist in neighbors for item in sublist])
    
    return E


print(build_edge_idx(3)[:, 0])
l, k = build_edge_idx(3)[:, 0]
print(l, k)

X = torch.tensor([l,k])
print(X)

tensor([0, 1])
tensor(0) tensor(1)
tensor([0, 1])


In [93]:
# Add the adjacency matrix as edge features

def adj_to_edge_attr(A, edge_index, edge_attr = None):
    """
    Converts a fully connected weighted adjacency matrix to edge features.
    
    
    Args:
        A (torch.Tensor): Adjacency matrix.
        edge_attr (torch.Tensor): Edge features of shape [num_edges, num_edge_features], optionally included.
    
    returns:
        edge_attr (torch.Tensor): New edge fatures of shape [num_edges, num_edge_features + 1].
    """

    num_nodes = A.shape[0]

    # If edge_attr is not included, initialize it
    if edge_attr is None:
        num_edges = int(num_nodes * (num_nodes - 1) / 2)
        edge_attr = torch.zeros((num_edges, 1))
        for i in range(num_edges):
            k, l = edge_index[:, i]
            edge_attr[i] = A[k, l]
    
        return edge_attr
    
    # TODO: Case when edge_attr is included
    else:
        return None

# Adj matrix
A = pdata_t[0][0][0]
num_nodes = A.shape[0]
print(A)
edge_index = build_edge_idx(num_nodes)
edge_attr = adj_to_edge_attr(A, edge_index)


tensor([[ 2.4411,  0.6607, -0.1258,  ...,  0.8382, -1.5899, -0.8058],
        [ 0.6607,  2.4411,  1.6718,  ...,  0.6053, -2.1600, -1.9578],
        [-0.1258,  1.6718,  2.4411,  ...,  0.4384, -1.4936, -1.5496],
        ...,
        [ 0.8382,  0.6053,  0.4384,  ...,  2.4411, -1.4207, -1.2728],
        [-1.5899, -2.1600, -1.4936,  ..., -1.4207,  2.4411,  1.8921],
        [-0.8058, -1.9578, -1.5496,  ..., -1.2728,  1.8921,  2.4411]],
       dtype=torch.float64)


In [94]:
num_nodes = A.shape[0]
complete_graph = build_edge_idx(num_nodes)

# Convert pdata_t from entries [[A, NF, EF], [A', NF', EF'], Y] to
# [[x', edge_index', edge_attr'], [x, edge_index, edge_attr], Y]
pdata_t2 = []
for i in range(len(pdata_t)):
    graph_1 = [pdata_t[i][0][0], complete_graph, adj_to_edge_attr(pdata_t[i][0][0], complete_graph)]
    graph_2 = [pdata_t[i][1][0], complete_graph, adj_to_edge_attr(pdata_t[i][1][0], complete_graph)]
    label = pdata_t[i][2]
    pdata_t2.append([graph_1, graph_2, label])

In [ ]:
# Link for pairs of graphs: https://pytorch-geometric.readthedocs.io/en/latest/advanced/batching.html
# Link for creating datasets: https://pytorch-geometric.readthedocs.io/en/latest/tutorial/create_dataset.html
# Link for Data handling tutorial: https://www.youtube.com/watch?v=Vz5bT8Xw6Dc&list=PLGMXrbDNfqTzqxB1IGgimuhtfAhGd8lHF&index=5
